In [2]:
import pandas as pd
import geopandas as gpd

## Load parks

In [3]:
df_parks = gpd.read_file("https://data.cityofnewyork.us/resource/enfh-gkve.geojson?$limit=1000000")

In [4]:
df_parks[df_parks['communityboard']=='103'].to_file('data/parks.geojson')

## Load subsidized housing

In [5]:
#df_subsidized = pd.read_csv("https://furmancenter.org/files/CoreData/FC_SHD_bbl_analysis_2024-08-29.csv")
df_subsidized = pd.read_csv("data/FC_SHD_bbl_analysis_2024-08-29.csv")
df_subsidized.columns[30:]

/var/folders/_q/g5l2s5950hz1zr9syt43gzp80000gn/T/ipykernel_39566/3682638536.py:2: DtypeWarning: Columns (103,106,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_subsidized = pd.read_csv("data/FC_SHD_bbl_analysis_2024-08-29.csv")


Index(['data_dof', 'data_hcrlihtc', 'data_hpd', 'data_hudcon', 'data_hudfin',
       'data_hudlihtc', 'data_ml', 'data_nycha', 'sub_fin', 'sub_prog',
       'sub_tax', 'sub_zone', 'prog_202_8', 'start_202_8', 'end_202_8',
       'prog_221d', 'start_221d', 'end_221d', 'prog_223', 'start_223',
       'end_223', 'prog_420c', 'start_420c', 'end_420c', 'prog_421a',
       'start_421a', 'end_421a', 'prog_421a_aff', 'start_421a_aff',
       'end_421a_aff', 'prog_8a', 'start_8a', 'end_8a', 'prog_hpd_oth',
       'start_hpd_oth', 'end_hpd_oth', 'prog_hud_fin_oth', 'start_hud_fin_oth',
       'end_hud_fin_oth', 'prog_hud_proj_oth', 'start_hud_proj_oth',
       'end_hud_proj_oth', 'prog_inclus_hous', 'start_inclus_hous',
       'end_inclus_hous', 'prog_j51', 'start_j51', 'end_j51', 'prog_lamp',
       'start_lamp', 'end_lamp', 'prog_lihtc4', 'start_lihtc4', 'end_lihtc4',
       'prog_lihtc9', 'start_lihtc9', 'end_lihtc9', 'prog_lmsa', 'start_lmsa',
       'end_lmsa', 'prog_mfp', 'start_mfp', 'end

In [6]:
# load mappluto
#pluto = gpd.read_file("data/nyc_mappluto_25v1_1_fgdb/MapPLUTO25v1_1.gdb")
#plutocb3 = pluto[pluto['CD']==103]
#plutocb3.to_file('data/cb3_pluto.geojson')
plutocb3 = gpd.read_file('data/cb3_pluto.geojson')

In [7]:
plutocb3_subhousing = plutocb3.merge(df_subsidized.rename(columns={'bbl':'BBL'}), on='BBL')

In [184]:
nycha = plutocb3_subhousing[(plutocb3_subhousing['prog_nycha_mix']==1)|(plutocb3_subhousing['prog_nycha_ph']==1)]
nycha['Address'] = nycha['Address'].str.strip()

/opt/anaconda3/envs/cb3-census/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [185]:
# load nycha development names
nycha_names = gpd.read_file("https://data.cityofnewyork.us/resource/3ub5-4ph8.geojson?$limit=100000")
nycha_names = nycha_names[(nycha_names['community_district'] == '3')&(nycha_names['borough'] == 'MANHATTAN')]
nycha_manual = pd.read_excel("data/nycha_development_names.xlsx")
nycha_names = pd.concat([nycha_names[['address', 'development']], nycha_manual], axis=0).drop_duplicates()
nycha_names['address'] = nycha_names['address'].str.strip()
nycha_mgd = nycha.merge(nycha_names[['address', 'development']].drop_duplicates(), right_on='address', left_on='Address', how='left')
nycha_mgd[['Address', 'development', 'geometry', 'OwnerName']].to_crs(4326).to_file('data/nycha.geojson')

In [9]:
sec8 = plutocb3_subhousing[(plutocb3_subhousing['prog_proj8']==1)|(plutocb3_subhousing['prog_rad']==1)]
sec8.to_crs(4326).to_file('data/sec8.geojson')

In [10]:
ml = plutocb3_subhousing[(plutocb3_subhousing['prog_ml']==1)]
ml.to_crs(4326).to_file('data/ml.geojson')

In [11]:
knickerbocker_village = plutocb3[plutocb3['BBL'].astype(str).str.contains('002530001')][['Address', 'OwnerName', 'geometry']]
knickerbocker_village.to_crs(4326).to_file('data/knickerbocker_village.geojson')

## Load Crashes

In [12]:
# load 2023 sectors
gdf_sectors = gpd.read_file("data/acs_2023_mapbox.geojson")

In [18]:
gdf_crash_2023 = gpd.read_file("https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$limit=10000000&$where=date_extract_y(crash_date)=2023")
gdf_crash_2024 = gpd.read_file("https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$limit=10000000&$where=date_extract_y(crash_date)=2024")
gdf_crash_2025 = gpd.read_file("https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$limit=10000000&$where=date_extract_y(crash_date)=2025")

Skipping field crash_time: unsupported OGR type: 10
Skipping field crash_time: unsupported OGR type: 10
Skipping field crash_time: unsupported OGR type: 10


In [22]:
gdf_crash = pd.concat([gdf_crash_2025, gdf_crash_2024, gdf_crash_2023])

In [32]:
# spatial join, add data columns
columns_to_drop = [x for x in gdf_sectors.columns if x in gdf_crash.columns and x != 'geometry']
gdf_crash_cb3 = gdf_crash.sjoin(gdf_sectors.drop(columns_to_drop, axis=1), how='inner')
gdf_crash_cb3['crash_date'] = pd.to_datetime(gdf_crash_cb3['crash_date'])
gdf_crash_cb3['crash_month'] = gdf_crash_cb3['crash_date'].dt.month
gdf_crash_cb3['crash_year'] = gdf_crash_cb3['crash_date'].dt.year
gdf_crash_cb3.head()

,location_state,number_of_motorist_injured,zip_code,number_of_cyclist_killed,location_zip,contributing_factor_vehicle_3,latitude,on_street_name,collision_id,number_of_persons_killed,...,with_income_diff,with_income_pct_change,with_income_pct_diff,with_income_pct_pct_change,nh_nhpi_pct_change,nh_nhpi_pct_pct_change,nhpi_pct_change,nhpi_pct_pct_change,totalpop_female_over85_pct_change,totalpop_female_over85_pct_pct_change
36,,0,10009,0,,None,40.730442,E 14 ST,4790150,0,...,329.0,0.023068,0.030792,0.035387,NaN,NaN,NaN,NaN,-0.201613,-0.203054
321,,1,10002,0,,None,40.713253,FRANKLIN D ROOSEVELT DR,4788959,0,...,1358.0,0.151259,-0.003800,-0.004247,NaN,NaN,NaN,NaN,-0.183044,-0.280930
351,,0,10002,0,,None,40.713554,MADISON ST,4793933,0,...,266.0,0.018433,0.046660,0.056551,NaN,NaN,NaN,NaN,-0.458861,-0.441016
508,,0,None,0,,None,40.708763,FDR DRIVE,4790508,0,...,-984.0,-0.083256,-0.024846,-0.029717,0.083333,0.139252,0.083333,0.139252,-0.170082,-0.127244
541,,1,10002,0,,None,40.71376,GRAND ST,4783783,0,...,266.0,0.018433,0.046660,0.056551,NaN,NaN,NaN,NaN,-0.458861,-0.441016


In [ ]:
collision_cols = ['number_of_motorist_injured', 'number_of_cyclist_killed', 
'number_of_cyclist_injured', 'number_of_persons_killed', 'number_of_persons_injured',
'number_of_pedestrians_killed', 'number_of_pedestrians_injured']
gdf_crash_cb3[collision_cols] = gdf_crash_cb3[collision_cols].astype(int)

In [92]:
# count number of collisions
collision_count = gdf_crash_cb3[['sectors', 'crash_month', 'crash_year']].value_counts().rename('collision_count').reset_index()
collision_count.head()

,sectors,crash_month,crash_year,collision_count
0,1,5,2025,21
1,1,4,2025,18
2,5,2,2025,13
3,9,1,2025,13
4,4,2,2025,13


In [93]:
gdf_crash_cb3_total = gdf_crash_cb3.groupby(['sectors', 'crash_month', 'crash_year'], \
    as_index=False)[collision_cols].sum().fillna(0)\
    .merge(collision_count, on=['sectors', 'crash_month', 'crash_year'], how='outer')
collision_cols = collision_cols + ['collision_count']
gdf_crash_cb3_total

,sectors,crash_month,crash_year,number_of_motorist_injured,number_of_cyclist_killed,number_of_cyclist_injured,number_of_persons_killed,number_of_persons_injured,number_of_pedestrians_killed,number_of_pedestrians_injured,collision_count
0,1,1,2025,1,0,1,0,4,0,2,11
1,1,2,2025,1,0,1,0,4,0,2,7
2,1,3,2025,2,0,1,0,3,0,0,6
3,1,4,2025,2,0,2,0,6,0,2,18
4,1,5,2025,1,0,4,0,12,0,6,21
5,10,1,2025,5,0,0,0,5,0,0,10
6,10,2,2025,0,0,0,0,0,0,0,4
7,10,3,2025,1,0,0,0,1,0,0,6
8,10,4,2025,8,0,1,0,10,0,1,11
9,10,5,2025,8,0,1,0,9,0,0,10


In [94]:
full_index = pd.MultiIndex.from_product([gdf_crash_cb3_total['sectors'].unique(), gdf_crash_cb3_total['crash_month'].unique(),
gdf_crash_cb3_total['crash_year'].unique()], names=['sectors', 'crash_month', 'crash_year'])

In [95]:
gdf_crash_cb3_complete = gdf_crash_cb3_total.set_index(['sectors', 'crash_month', 'crash_year']).reindex(full_index).reset_index()
gdf_crash_cb3_complete = gdf_crash_cb3_complete[~((gdf_crash_cb3_complete['crash_month'] > 5) & (gdf_crash_cb3_complete['crash_year'] == 2025))]
gdf_crash_cb3_complete[collision_cols] = gdf_crash_cb3_complete[collision_cols].fillna(0)
gdf_crash_cb3_complete.shape, gdf_crash_cb3_total.shape

((50, 11), (50, 11))

In [96]:
gdf_crash_cb3_summary = gdf_crash_cb3_total.groupby('sectors')[collision_cols].mean()
gdf_crash_cb3_summary

,number_of_motorist_injured,number_of_cyclist_killed,number_of_cyclist_injured,number_of_persons_killed,number_of_persons_injured,number_of_pedestrians_killed,number_of_pedestrians_injured,collision_count
sectors,,,,,,,,
1,1.4,0.0,1.8,0.0,5.8,0.0,2.4,12.6
10,4.4,0.0,0.4,0.0,5.0,0.0,0.2,8.2
2,1.6,0.0,1.2,0.0,4.6,0.0,1.8,8.4
3,1.8,0.0,0.8,0.0,4.0,0.0,1.4,8.0
4,0.6,0.0,1.2,0.0,2.2,0.0,0.2,7.0
5,3.2,0.0,0.4,0.0,5.2,0.0,1.6,9.2
6,1.6,0.0,0.4,0.0,3.4,0.0,1.2,10.6
7,3.8,0.0,0.2,0.0,5.0,0.0,1.0,8.2
8,0.4,0.0,1.0,0.0,2.6,0.0,1.2,8.0


In [97]:
gdf_sectors.drop(columns_to_drop, axis=1).merge(gdf_crash_cb3_summary, on='sectors').to_file("data/acs_2023_mapbox.geojson")